## I. 데이터 불러오고, 라벨링 및 특징 확인
먼저 주어진 학습데이터와 테스트데이터를 불러오고, 라벨링합니다. 그 후 주어진 데이터의 특징을 파악해봅니다.

### 1. 데이터 불러오기

라벨링 과정에서 `nltk.bigrams()`, `nltk.trigrams()` 등을 활용해 목표 단어 좌우의 맥락을 고려할 수도 있으나, 처리할 데이터의 양이 최소 2배 이상 많아진다는 단점이 존재합니다.


따라서 본 과제에서는 전후맥락을 고려하지 않고, 단순히 단어 하나하나에만 집중해 라벨링합니다.

<br>

더불어, 원칙적으로는 주어진 데이터를 무작위로 섞은 뒤(shuffling), 모델을 학습시키기 위한 training set, 정확도를 개선시키기 위한 validation set, 그리고 모델의 정확도를 측정하기 위한 test set을 구분하는 과정을 거쳐야합니다. 이는 모델이 학습한 데이터에 대해서만 정확도가 높게 나오는 경우를 방지하기 위함입니다.

**그러나 본 과제에서는 이미 train set과 test set이 구분되어 있고, validation set은 고려하지 않으므로 이러한 절차를 생략합니다.**

In [1]:
import nltk, re, os
import Hangul as hg

## read로 불러온 raw
raw_train = open('train.txt', 'r', encoding='utf8').read()
raw_test = open('test.txt', 'r', encoding='utf8').read()


## 맥락무시하고 라벨링
labeled_train = re.findall('\d+\t(.+?)\t(.+?)\n', raw_train)
labeled_test = re.findall('\d+\t(.+?)\t(.+?)\n', raw_test)

print(labeled_test[:10])

[('상주시', 'LOC'), ('대중지', '-'), ('선’은', 'ORG'), ('배당률을', '-'), ('매기며', '-'), ('유력한', '-'), ('차기', '-'), ('풋내기들을', 'CVL'), ('열거했다', '-'), ('.', '-')]


### 2. 데이터 특성 파악하기

우선 주어진 학습용데이터를 살펴봅니다.

원본 데이터셋 텍스트파일의 구성형식은 다음과 같습니다.
```
1	22회말	NUM
2	21-21	NUM
3	동점타의	-
4	꼬냑인	CVL
5	LG	ORG
6	막둥이	CVL
7	이종열은	PER
8	"팀이	-
9	어려울	-
10	때	-
11	귀중한	-
12	타점을	-
13	올려	-
14	원기가	-
15	좋다	-
16	.	-
```

주어진 데이터들에 대하여 `nltk.FreqDist()`함수를 활용해 태그들의 분포를 확인합니다.
```
('-', 437495), ('NUM', 38910), ('CVL', 36946), ('PER', 28919), ('ORG', 27218), ('DAT', 20355), ('TRM', 13185), ('LOC', 12722), ('EVT', 10401), ('ANM', 3935), ('AFW', 3618), ('TIM', 2690), ('FLD', 1437), ('PLT', 180), ('MAT', 128)
```
비개채명 태그(-)가 가장 많고, 이후 숫자(NUM), 문명어(CVL), 인명(PER), 단체(ORG) 순으로 태그가 많이 분포하는 것을 확인할 수 있었습니다.

따라서 숫자, 문명어 등 빈번히 등장하는 태그들을 특히 유의하며 과제를 진행합니다.

In [2]:
print(nltk.FreqDist(tag for (_, tag) in labeled_train).most_common(50))

[('-', 437495), ('NUM', 38910), ('CVL', 36946), ('PER', 28919), ('ORG', 27218), ('DAT', 20355), ('TRM', 13185), ('LOC', 12722), ('EVT', 10401), ('ANM', 3935), ('AFW', 3618), ('TIM', 2690), ('FLD', 1437), ('PLT', 180), ('MAT', 128)]


# II. 데이터 전처리(조사 및 좌우 문장부호 제거)
본 데이터는 기본적으로 마침표가 아닌 문장부호는 하나의 단어로 인식합니다.

따라서 `토끼`, `토끼,`, `"토끼`처럼 내용상으로는 같은 단어도, 문장부호 유무에 따라서 다른 단어로 인식하는 경우가 발생할 수 있습니다. 또한 `토끼를`, `토끼의`, `토끼는`처럼단어 뒤에 조사가 붙어있는 경우에도, 내용상 같은 단어를 다르게 인식할 수 있습니다.

따라서 본격적으로 특징추츨하기 전, 다음과 같은 조사 목록과 함수를 통하여 단어 좌우에 붙은 문장부호와 조사를 제거하는 절차를 진행했습니다. 

조사 목록은 [이기창님의 블로그](https://ratsgo.github.io/korean%20linguistics/2017/03/15/words/)를 참고하여 구축하였습니다.

In [5]:
## 빈도수에 따른 조사목록 구축
particle_list = [
    ##빈도수 > 2000000
    '[에]?(은|는)', '(을|를)', '(이|가)', '[와과]?의', '에', '(로|으로)', 
    ## 빈도수 > 500000
    '(와|과)', '도', '에서', '[이]?[지]?만', 
    ## 빈도수 > 50000
    '(이나|나)', '까지', '부터', '(에게|께)', '보다', '처럼', 
    '(이라도|라도)', '(으로서|로서)', '조차', '만큼', '같이', '마저', '(이나마|나마)', '(한테|더러)',
    '(에게서|한테서|께서)', '이야', '이라[야고]?',
]


## 단어 좌우의 문장부호를 제거하는 함수
def word_stripper(word):
    try:
        return re.search('\\b(.+)\\b', word).group(1)
    except:
        return word


## 문장부호를 제거한 뒤, 조사목록을 활용해 조사를 제거하는 함수    
def tail_maker(word):
    
    s_word = word_stripper(word)
    
    for particle in particle_list:
        if bool(re.search('[들]?'+particle + '$', s_word)) == True:
            return particle
        
    return ''


## 예시
for word in ['토끼를', '토끼는', '토끼가']:
    print(tail_maker(word), word.rstrip(tail_maker(word)))

(을|를) 토끼
[에]?(은|는) 토끼
(이|가) 토끼


조사와 문장부호만 제거한 단어를 특징삼아 나이브베이즈분류기 모델을 돌려봅니다.

결과적으로 학습데이터에 대하여 약 88.68%의 정확도를 확인할 수 있었습니다.

테스트데이터를 활용해 정확도를 측정할 시, 정확도가 더 낮아질 가능성이 높으므로 더욱 성능을 향상시켜봅시다.

In [6]:
def feature_extraction(word):
    features = {}
    
    s_word = word_stripper(word)
    word_tail = tail_maker(s_word)
    t_word = s_word.rstrip(word_tail)
    
    ## 양 끝 문장부호, 조사 분리
    features['particle'] = word_tail
    features['untail_word'] = t_word
    
    return features

print(feature_extraction("'토끼는,"))

{'particle': '[에]?(은|는)', 'untail_word': '토끼'}


In [7]:
featured_train = [(feature_extraction(word), label)
                    for (word, label) in labeled_train]

featured_test = [(feature_extraction(word), label)
                    for (word, label) in labeled_test]

print(featured_train[:1])


## 베이즈확률 계산하기
classifier = nltk.NaiveBayesClassifier.train(featured_train)
print(nltk.classify.accuracy(classifier, featured_train))

classifier.show_most_informative_features(50)

[({'particle': '', 'untail_word': '비토리오'}, 'PER')]
0.8868726092591113
Most Informative Features
             untail_word = '.'                 - : AFW    =   4729.3 : 1.0
             untail_word = '기자'              CVL : -      =   1256.8 : 1.0
             untail_word = '지난'              DAT : ORG    =   1039.9 : 1.0
             untail_word = '10년'             DAT : -      =    997.8 : 1.0
             untail_word = '한'               NUM : CVL    =    973.8 : 1.0
             untail_word = 'SK'              ORG : -      =    906.9 : 1.0
             untail_word = 'press@mydaily.co.kr모바일'    TRM : -      =    839.3 : 1.0
             untail_word = '1일'              DAT : -      =    809.7 : 1.0
             untail_word = '6일'              DAT : -      =    799.3 : 1.0
             untail_word = '감독'              CVL : -      =    738.0 : 1.0
             untail_word = '5일'              DAT : -      =    701.8 : 1.0
             untail_word = '1차전'             EVT : -      =    644.9 

## III. 태그별로 특징 추출

정확도를 더 개선하고자, 특정 태그들을 대상으로 특징을 추출해봅니다.

### 1. NUM 태그

먼저 비개체명 태그(-) 다음으로 빈번히 출현했던 NUM 태그에 대한 특징을 설정해봅니다.

<br>

가장 먼저 아라비아 숫자(1,2,3)이 포함되지 여부를 확인하는 `contain_digit` 특징을 추가해, 다시 한 번 모델을 돌려봅니다.

그 결과, 학습데이터에 대하여 약 88.40%로 이전보다 약간 낮은 정확도를 확인할 수 있었습니다.

In [8]:
def feature_extraction(word):
    features = {}
    
    s_word = word_stripper(word)
    word_tail = tail_maker(s_word)
    t_word = s_word.rstrip(word_tail)
    
    ## 양 끝 문장부호, 조사 분리
    features['untail_word'] = t_word
    features['particle'] = word_tail

    ## NUM tag
    features['contain_digit'] = any(i.isdigit() for i in t_word)


    return features

for word in ['1위는', '한사람','1마리','2일']:
    print(feature_extraction(word))

{'untail_word': '1위', 'particle': '[에]?(은|는)', 'contain_digit': True}
{'untail_word': '한사람', 'particle': '', 'contain_digit': False}
{'untail_word': '1마리', 'particle': '', 'contain_digit': True}
{'untail_word': '2일', 'particle': '', 'contain_digit': True}


In [9]:
featured_train_data = [(feature_extraction(word), label)
                    for (word, label) in labeled_train]

featured_test_data = [(feature_extraction(word), label)
                    for (word, label) in labeled_test]

classifier = nltk.NaiveBayesClassifier.train(featured_train_data)
print(nltk.classify.accuracy(classifier, featured_train_data))
classifier.show_most_informative_features(100)


0.8840926506607495
Most Informative Features
             untail_word = '.'                 - : AFW    =   4729.3 : 1.0
             untail_word = '기자'              CVL : -      =   1256.8 : 1.0
             untail_word = '지난'              DAT : ORG    =   1039.9 : 1.0
             untail_word = '10년'             DAT : -      =    997.8 : 1.0
             untail_word = '한'               NUM : CVL    =    973.8 : 1.0
             untail_word = 'SK'              ORG : -      =    906.9 : 1.0
             untail_word = 'press@mydaily.co.kr모바일'    TRM : -      =    839.3 : 1.0
             untail_word = '1일'              DAT : -      =    809.7 : 1.0
             untail_word = '6일'              DAT : -      =    799.3 : 1.0
             untail_word = '감독'              CVL : -      =    738.0 : 1.0
             untail_word = '5일'              DAT : -      =    701.8 : 1.0
             untail_word = '1차전'             EVT : -      =    644.9 : 1.0
             untail_word = '한국'              

NUM 태그와 관련하여 잘못분류된 경우를 살펴봅니다.

한, 둘처럼 아라비아 숫자가 아닌 NUM 태그들이 잘못분류된 경우가 많은 것을 확인할 수 있었습니다.

In [11]:
errors = []

for (word, tag) in labeled_train[:1000000]:
    guess = classifier.classify(feature_extraction(word))
    if guess != tag and tag == "NUM":
        errors.append((guess, tag, word))
        
for (guess, tag, word) in sorted(errors):
    print(f'guess: {guess:<5} tag: {tag:<5} word: {word:<15}')

guess: -     tag: NUM   word: "둘이서           
guess: -     tag: NUM   word: "둘이서           
guess: -     tag: NUM   word: "몇몇            
guess: -     tag: NUM   word: "양팀            
guess: -     tag: NUM   word: "일대일           
guess: -     tag: NUM   word: "제             
guess: -     tag: NUM   word: "한             
guess: -     tag: NUM   word: '더블'의          
guess: -     tag: NUM   word: '두마리           
guess: -     tag: NUM   word: '방어율           
guess: -     tag: NUM   word: '쌍포'가          
guess: -     tag: NUM   word: '톱             
guess: -     tag: NUM   word: '톱             
guess: -     tag: NUM   word: '톱             
guess: -     tag: NUM   word: '톱             
guess: -     tag: NUM   word: '톱             
guess: -     tag: NUM   word: '톱             
guess: -     tag: NUM   word: '톱             
guess: -     tag: NUM   word: '톱             
guess: -     tag: NUM   word: '톱             
guess: -     tag: NUM   word: '한             
guess: -     tag: NUM   word: (G) 

### 2. 아라비아 숫자가 포함되지 않은 NUM_tag

앞선 분석 결과에 따라서, 아라비아가 아닌 NUM 태그들에 해당하는 목록을 구축합니다.

이후 해당 목록에 있는 단어들이 포함되어있는지 여부를 새로운 특징으로 추가합니다.

In [12]:
non_digit_NUM = [
    '^하나$', '^둘[^러레]', '^셋', '^넷', 
    '^한$', '^두$', '^세$', '^네$',
    '다섯', '여섯', '일곱', '여덟', '아홉',
    '한번', '두번', '세번', '네번', '다섯번', '여섯번', '아홉번', '열번',
    '첫', '몇', '^양[팀국]$', '^양$'
    '수[십백천억]',
    ##스포츠 관련
    '버디', '보기', '이글', 'WHIP', '볼넷',
    ##기타
    '^번[씩]?$','절반'
]

def contain(word, x_list):
    for x in x_list:
        if bool(re.search(x, word)) == True:
            return x
    else:
        return False
    
## 예시
for word in ['한화', '수십명', '한번에', '다섯번']:
    print(contain(word, non_digit_NUM))

False
False
한번
다섯


이후 해당 특징을 추가하여 다시 한번 모델을 돌려봅니다.

그 결과, 학습데이터에 대하여 약 88.56%의 정확도로, 아라비아 숫자의 유무만을 특징으로 설정했을 때보다 0.16%p 정확도가 향상된 것을 확인할 수 있었습니다.

In [14]:
def feature_extraction(word):
    features = {}
    
    s_word = word_stripper(word)
    word_tail = tail_maker(s_word)
    t_word = s_word.rstrip(word_tail)
    
    ## 양 끝 문장부호, 조사 분리
    features['untail_word'] = t_word
    features['particle'] = word_tail

    ## NUM tag
    features['contain_digit'] = any(i.isdigit() for i in t_word)
    features['contain_non_digit_NUM?'] = contain(t_word, non_digit_NUM)


    return features


featured_train_data = [(feature_extraction(word), label)
                    for (word, label) in labeled_train]

featured_test_data = [(feature_extraction(word), label)
                    for (word, label) in labeled_test]

print(featured_train_data[:1])




classifier = nltk.NaiveBayesClassifier.train(featured_train_data)
print(nltk.classify.accuracy(classifier, featured_train_data))
classifier.show_most_informative_features(500)


[({'untail_word': '비토리오', 'particle': '', 'contain_digit': False, 'contain_non_digit_NUM?': False}, 'PER')]
0.8856675426513659
Most Informative Features
             untail_word = '.'                 - : AFW    =   4729.3 : 1.0
             untail_word = '기자'              CVL : -      =   1256.8 : 1.0
             untail_word = '지난'              DAT : ORG    =   1039.9 : 1.0
  contain_non_digit_NUM? = '^두$'             NUM : -      =   1026.4 : 1.0
             untail_word = '10년'             DAT : -      =    997.8 : 1.0
             untail_word = '한'               NUM : CVL    =    973.8 : 1.0
  contain_non_digit_NUM? = '^한$'             NUM : CVL    =    940.4 : 1.0
             untail_word = 'SK'              ORG : -      =    906.9 : 1.0
             untail_word = 'press@mydaily.co.kr모바일'    TRM : -      =    839.3 : 1.0
             untail_word = '1일'              DAT : -      =    809.7 : 1.0
             untail_word = '6일'              DAT : -      =    799.3 : 1.0
            

NUM, DAT, TIM 태그 중 잘못 분류된 태그들을 확인해봅니다.

DAT와 TIM태그가 NUM태그로 잘못 분류된 경우가 많은 것을 확인할 수 있습니다.

In [15]:
errors = []

for (word, tag) in labeled_train:
    guess = classifier.classify(feature_extraction(word))
    if guess != tag and tag in ['NUM', 'DAT', 'TIM']:
        errors.append((guess, tag, word))
        
nltk.ConditionalFreqDist((guess, tag) for (guess, tag, word) in errors).tabulate()

     DAT  NUM  TIM 
  - 4428 5140 1017 
ANM    0    3    0 
CVL   10   54    0 
DAT    0   49   19 
EVT   10   17    1 
LOC    8   10    0 
NUM  894    0  624 
ORG   29   41    3 
PER    2    3    1 
TIM    6    2    0 
TRM    3  184    0 


### 3. TIM/DAT 태그

역시 DAT, TIM 태그들에 해당하는 단어들의 목록을 구축해, 특징으로 추가해 모델을 돌려봅니다.

그 결과, 학습데이터에 대하여 약 88.54%의 정확도를 얻을 수 있었습니다.

In [18]:
DAT_list = [
    '[\d차예내작매수금전다금당명익본양][개주]?[년월일]', '\d[/]', '\d{4}', '\d세기', '\d분기',
    '연[간내말초]',
    ##특수한 단어
    '^지난', '다음', '이번', '예전', '옛날', '전[날에]', 
    '오늘', '내일', '날짜', '월경기', '^동안', '유행기', '방학', '일제강점기', '하계',
    '호시기', '휴가철',
    ## 한글표현
    '하루', '이틀','사흘','나흘', '열흘', '며칠', '나흗날', '오뉴월', '한달',
    '이듬해', '올해', '^해$', '^달$', '^주$',
    ## 계절
    '봄', '(여름|서머)', '가을', '겨울', '(계절|시즌)',
    ## 요일
    '[월화수목금토일]요일', '공휴일', '일주간', '일주일', '주말', '주중','일일',
]

In [19]:
def contain(word, x_list):
    for x in x_list:
        if bool(re.search(x, word)) == True:
            return x
    else:
        return False


def feature_extraction(word):
    features = {}
    
    s_word = word_stripper(word)
    word_tail = tail_maker(s_word)
    t_word = s_word.rstrip(word_tail)
    
    ## 양 끝 문장부호, 조사 분리
    features['untail_word'] = t_word
    features['particle'] = word_tail

    ## NUM tag
    features['contain_digit'] = any(i.isdigit() for i in t_word)
    features['contain_non_digit_NUM?'] = contain(t_word, non_digit_NUM)
    
    ## DAT tag
    features['contain_DAT'] = contain(t_word, DAT_list)

    return features

feature_extraction('12일에')


{'untail_word': '12일',
 'particle': '에',
 'contain_digit': True,
 'contain_non_digit_NUM?': False,
 'contain_DAT': '[\\d차예내작매수금전다금당명익본양][개주]?[년월일]'}

In [20]:
featured_train_data = [(feature_extraction(word), label)
                    for (word, label) in labeled_train]

featured_test_data = [(feature_extraction(word), label)
                    for (word, label) in labeled_test]

print(featured_train_data[:1])


classifier = nltk.NaiveBayesClassifier.train(featured_train_data)
print(nltk.classify.accuracy(classifier, featured_train_data))
classifier.show_most_informative_features(100)

[({'untail_word': '비토리오', 'particle': '', 'contain_digit': False, 'contain_non_digit_NUM?': False, 'contain_DAT': False}, 'PER')]
0.8854152465215259
Most Informative Features
             contain_DAT = '\\d[/]'          DAT : -      =   6819.9 : 1.0
             untail_word = '.'                 - : AFW    =   4729.3 : 1.0
             contain_DAT = '[\\d차예내작매수금전다금당명익본양][개주]?[년월일]'    DAT : TRM    =   1448.7 : 1.0
             contain_DAT = '^지난'             DAT : ORG    =   1382.7 : 1.0
             untail_word = '기자'              CVL : -      =   1256.8 : 1.0
             untail_word = '지난'              DAT : ORG    =   1039.9 : 1.0
  contain_non_digit_NUM? = '^두$'             NUM : -      =   1026.4 : 1.0
             untail_word = '10년'             DAT : -      =    997.8 : 1.0
             untail_word = '한'               NUM : CVL    =    973.8 : 1.0
  contain_non_digit_NUM? = '^한$'             NUM : CVL    =    940.4 : 1.0
             untail_word = 'SK'              ORG : -     

### 4. 처음 n 글자

앞에서부터 n번째 글자를 특징으로 추가해봅니다.

글자수에 따른 정확도 변화는 다음과 같습니다.

|글자수|정확도|
|---|---|
|1|90.45%|
|2|93.18%|
|3|95.24%|
|4|95.61%|

세번째 글자보다 글자수가 늘어날 경우 정확도가 상승하는 것처럼 보이지만, 이는 테스트데이터에서는 되려 오버피팅 현상을 일으킬 수 있다고 판단했습니다.

따라서 최종모델에서는 처음의 2글자만 활용해 특징을 설정하는 것으로 결정했습니다.

In [27]:
def contain(word, x_list):
    for x in x_list:
        if bool(re.search(x, word)) == True:
            return x
    else:
        return False

def feature_extraction(word):
    features = {}
    
    s_word = word_stripper(word)
    word_tail = tail_maker(s_word)
    t_word = s_word.rstrip(word_tail)
    
    ## 양 끝 문장부호, 조사 분리
    features['untail_word'] = t_word
    features['particle'] = word_tail

    ## NUM tag
    features['contain_digit'] = any(i.isdigit() for i in t_word)
    features['contain_non_digit_NUM?'] = contain(t_word, non_digit_NUM)
    
    ## DAT tag
    features['contain_DAT'] = contain(t_word, DAT_list)
    
    ## 처음 n글자
    features['first_n_letter'] = word[:n_letter]

    return features


for n_letter in range(1,5):
    featured_train_data = [(feature_extraction(word), label)
                    for (word, label) in labeled_train]

    featured_test_data = [(feature_extraction(word), label)
                    for (word, label) in labeled_test]

    classifier = nltk.NaiveBayesClassifier.train(featured_train_data)
    print(nltk.classify.accuracy(classifier, featured_train_data))


0.9045411736314501
0.9318784778864793
0.9524554995071607
0.9561835274133065


## IV. 최종 모델 구축

이상의 결과들을 종합하여, 다음과 같은 특징들을 활용해 최종모델을 구축하였습니다. (괄호 안에는 `해당 특징`에 대응하는 변수명을 적어두었습니다.)

+ 조사와 문장부호 처리
  + 조사 및 문장부호를 제거한 단어(`untail_word`) 
  + 전처리하기 전 원래 붙어있던 조사(`particle`)
+ NUM 태그 처리
  + 아라비아 숫자를 포함하는지 여부(`contain_digit`)
  + 아라비아 숫자가 아닌 숫자어를 포함하는지 여부(`contain_non_digit_NUM?`)
+ DAT 태그 처리
  + DAT 태그 목록에 해당하는 단어를 포함하는지 여부(`contain_DAT`)
+ 처음 2글자 단어(`first_2_letter`)

이상의 특징들을 활용해 테스트데이터에 대하여 정확도를 측정한 결과, 최종적으로 약 88.15%의 정확도를 확인할 수 있었습니다.

In [29]:
non_digit_NUM = [
    '^하나$', '^둘[^러레]', '^셋', '^넷', 
    '^한$', '^두$', '^세$', '^네$',
    '다섯', '여섯', '일곱', '여덟', '아홉',
    '한번', '두번', '세번', '네번', '다섯번', '여섯번', '아홉번', '열번',
    '첫', '몇', '^양[팀국]$', '^양$'
    '수[십백천억]',
    ##스포츠 관련
    '버디', '보기', '이글', 'WHIP', '볼넷',
    ##?
    '^번[씩]?$','절반'
]


DAT_list = [
    '[\d차예내작매수금전다금당명익본양][개주]?[년월일]', '\d[/]', '\d{4}', '\d세기', '\d분기',
    '연[간내말초]',
    ##특수한 단어
    '^지난', '다음', '이번', '예전', '옛날', '전[날에]', 
    '오늘', '내일', '날짜', '월경기', '^동안', '유행기', '방학', '일제강점기', '하계',
    '호시기', '휴가철',
    ## 한글표현
    '하루', '이틀','사흘','나흘', '열흘', '며칠', '나흗날', '오뉴월', '한달',
    '이듬해', '올해', '^해$', '^달$', '^주$',
    ## 계절
    '봄', '(여름|서머)', '가을', '겨울', '(계절|시즌)',
    ## 요일
    '[월화수목금토일]요일', '공휴일', '일주간', '일주일', '주말', '주중','일일',
    ## 수식어
    '^오는$', '^올$',
]

non_digit_NUM = [
    '^하나$', '^둘[^러레]', '^셋', '^넷', 
    '^한$', '^두$', '^세$', '^네$',
    '다섯', '여섯', '일곱', '여덟', '아홉',
    '한번', '두번', '세번', '네번', '다섯번', '여섯번', '아홉번', '열번',
    '첫', '몇', '^양[팀국]$', '^양$'
    '수[십백천억]',
    ##스포츠 관련
    '버디', '보기', '이글', 'WHIP', '볼넷',
    ##기타
    '^번[씩]?$','절반'
]


def contain(word, x_list):
    for x in x_list:
        if bool(re.search(x, word)) == True:
            return x
    else:
        return False


def feature_extraction(word):
    features = {}
    
    s_word = word_stripper(word)
    word_tail = tail_maker(s_word)
    t_word = s_word.rstrip(word_tail)
    
    ## 양 끝 문장부호, 조사 분리
    features['untail_word'] = t_word
    features['particle'] = word_tail

    ## NUM tag
    features['contain_digit'] = any(i.isdigit() for i in t_word)
    features['contain_non_digit_NUM?'] = contain(t_word, non_digit_NUM)
    
    ## DAT tag
    features['contain_DAT'] = contain(t_word, DAT_list)
    
    ## 처음 n글자
    features['first_2_letter'] = word[:2]

    return features


featured_train_data = [(feature_extraction(word), label)
                    for (word, label) in labeled_train]

featured_test_data = [(feature_extraction(word), label)
                    for (word, label) in labeled_test]

print(featured_train_data[:3])


classifier = nltk.NaiveBayesClassifier.train(featured_train_data)
print(nltk.classify.accuracy(classifier, featured_test_data))
classifier.show_most_informative_features(500)


[({'untail_word': '비토리오', 'particle': '', 'contain_digit': False, 'contain_non_digit_NUM?': False, 'contain_DAT': False, 'first_3_letter': '비토'}, 'PER'), ({'untail_word': '양일', 'particle': '', 'contain_digit': False, 'contain_non_digit_NUM?': False, 'contain_DAT': '[\\d차예내작매수금전다금당명익본양][개주]?[년월일]', 'first_3_letter': '양일'}, 'DAT'), ({'untail_word': '만', 'particle': '에', 'contain_digit': False, 'contain_non_digit_NUM?': False, 'contain_DAT': False, 'first_3_letter': '만에'}, '-')]
0.8815932981063954
Most Informative Features
             contain_DAT = '\\d[/]'          DAT : -      =   6819.7 : 1.0
             untail_word = '.'                 - : AFW    =   4729.3 : 1.0
          first_3_letter = '19'              NUM : -      =   4585.6 : 1.0
          first_3_letter = '24'              NUM : -      =   4043.7 : 1.0
          first_3_letter = '21'              NUM : -      =   3876.6 : 1.0
          first_3_letter = '12'              DAT : -      =   3606.7 : 1.0
          first_3_letter